# Swanson's ABC closed discovery model

## Motivation

The main idea behind Swanson's work is that the two originally unconnected parts of the literature (C and A) can establish latent connections (through intermediate terms in B). These two pieces of literature have one or more b-terms in common, creating an indirect connection. Once these bridges are identified and validated, new knowledge can be generated.

Swanson founded his discovery on the so-called ABC model, which is now widely regarded as a typical paradigm for LBD. The ABC model contains three concepts: the start concept ($c$), the intermediate concept ($b$), and the target concept ($a$). The LBD process begins with retrieving $c-b$ and $b-a$ relationships. Next, it combines associations with the same intermediates. Finally, it retrieves a list of $a-c$ relationships. If there is no prior mention of a particular $a-c$ connection, a hypothesis of a potential novel relationship between $a$ and $c$ concepts is conceived.

<img src="img/closed_discovery.png" alt="Swanson's ABC model" width="300px"/>

In Swanson's scenario from 1986, literature domain $C$ refers to Raynaud's disease (set of papers mentioning Raynaud's disease), whereas domain $A$ represents the fish oil literature (set of papers mentioning fish oil).

[spaCy](https://spacy.io) is a powerful NLP library that offers a wide range of built-in functionalities for various NLP tasks. It has become an industry standard, with a vast ecosystem of extensions and plugins that can be easily integrated into custom components and machine learning workflows.

In this notebook, we will use `en_core_web_md`, an English pipeline trained on written web text (blogs, news, comments). This pipeline includes vocabulary, syntax, and entities, and is optimized for CPU use. You can download the pipeline using the `spacy download` command.

Note: If you are in a Jupyter notebook, you should use the `!` prefix to execute the following command. Make sure to restart your kernel or runtime after installation to make sure that the installed pipeline package can be found.

In [3]:
# Load required libraries for NLP and data analysis
import gzip
import numpy as np
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Download the small SpaCy English NLP model
# !python -m spacy download en_core_web_sm

In [5]:
# Load the SpaCy model into the `nlp` pipeline for NLP processing
nlp = spacy.load('en_core_web_md')

In [6]:
# Load datasets for the two domains (Raynaud's disease and Fish oil literature)
# The datasets are stored as pipe-separated value (PSV) compressed files.
# Only PMID and title columns are used for subsequent processing.
c_df = pd.read_csv('./input/pmid_dp_ti_ab_raynaud_20241127.psv.gz', sep='|', names=['pmid', 'dp', 'ti', 'ab'], usecols=['pmid', 'ti'])
a_df = pd.read_csv('./input/pmid_dp_ti_ab_fish_20241127.psv.gz', sep='|', names=['pmid', 'dp', 'ti', 'ab'], usecols=['pmid', 'ti'])

In [7]:
# Number of documents in domains
print(f'No. of documents in domain C: {c_df.shape[0]}')
print(f'No. of documents in domain A: {a_df.shape[0]}')

No. of documents in domain C: 1273
No. of documents in domain A: 153


Before proceeding with further processing, we will remove the string '(author's transl)' from record titles originally published in non-English languages. For example, consider rows 126 (PMID 507701) and 128 (PMID 6783785) from the data frames `c_df` and `a_df` respectively. To simplyfy text processing we will employ `str.replace` method and remove '(author's transl)' string from titles where appropriate.

In [9]:
# Inspecting an example before cleaning
print(f"C: 128: {c_df.loc[128, 'ti']}")
print(f"A: 126: {a_df.loc[126, 'ti']}")

# Remove the translation marker from titles
c_df['ti'] = c_df['ti'].str.replace(" (author's transl)", '')
a_df['ti'] = a_df['ti'].str.replace(" (author's transl)", '')

# Inspecting the titles again after cleaning
print()
print(f"C: 128: {c_df.loc[128, 'ti']}")
print(f"A: 126: {a_df.loc[126, 'ti']}")

C: 128: [Surgery of thoracic outlet syndromes with Raynaud's disease. Report of 20 cases (author's transl)].
A: 126: [Nutritional lipidic factors of enzymatic phenobarbital induction in the rat. Effects of quantitative and qualitative composition of dietary lipids (author's transl)].

C: 128: [Surgery of thoracic outlet syndromes with Raynaud's disease. Report of 20 cases].
A: 126: [Nutritional lipidic factors of enzymatic phenobarbital induction in the rat. Effects of quantitative and qualitative composition of dietary lipids].


In [10]:
# Expand stop words with domain-specific PubMed stopwords

# SpaCy has a built-in list of stopwords, but domain-specific datasets like PubMed
# require additional stopwords to improve filtering during NLP tasks.

# Get the current count of SpaCy's default stopwords
print(f"Default stopwords count: {len(nlp.Defaults.stop_words)}")

stopwords_pubmed = set(open('./input/stopwords_pubmed', 'r').read().splitlines())
nlp.Defaults.stop_words |= stopwords_pubmed

# Check the updated count of stopwords
print(f"Updated stopwords count: {len(nlp.Defaults.stop_words)}")

Default stopwords count: 326
Updated stopwords count: 443


In [11]:
# --------------------------------------------------------------
# NLP tokenization and metadata extraction
# --------------------------------------------------------------

# Define helper functions to extract token-level details using SpaCy.
# The `extract_tokens_plus_meta` function extracts metadata for each token,
# and `tidy_tokens` aggregates these details into a tidy format for analysis.

def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct) for i in doc
    ]

def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "pmid", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct"
    ]
    
    ti2pmid_lst = docs.reindex(columns=['ti', 'pmid']).to_records(index=False,).tolist()
    
    meta_df = []
    for ti, pmid in nlp.pipe(ti2pmid_lst, as_tuples=True):
        meta = extract_tokens_plus_meta(ti)
        meta = pd.DataFrame(meta)
        meta.columns = cols[1:]
        meta = meta.assign(pmid = pmid).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df)

In [12]:
# Apply tokenization and tidy the metadata for both datasets
c_tbl = tidy_tokens(c_df)
a_tbl = tidy_tokens(a_df)

a_tbl.head()

,pmid,token,token_order,lemma,ent_type,tag,dep,pos,is_stop,is_alpha,is_digit,is_punct
0,91820,Fish,0,fish,,NN,compound,NOUN,False,True,False,False
1,91820,oils,1,oil,,NNS,ROOT,NOUN,False,True,False,False
2,91820,",",2,",",,",",punct,PUNCT,False,False,False,True
3,91820,prostaglandins,3,prostaglandin,,NNS,conj,NOUN,False,True,False,False
4,91820,",",4,",",,",",punct,PUNCT,False,False,False,True


In [13]:
# --------------------------------------------------------------
# Filtering Tokens for Meaningful Features
# --------------------------------------------------------------

# Filter out irrelevant tokens (stopwords, digits, and punctuation).
# Aggregate tokens by their lemma for each PMID.
c_lem_tbl = (c_tbl.query("is_stop == False & is_alpha == True & is_digit == False & is_punct == False")
             .sort_values(by=['pmid', 'token_order'])
             .groupby('pmid')
             .agg({'lemma': lambda x: ' '.join(x)})
             .reset_index())

a_lem_tbl = (a_tbl.query("is_stop == False & is_alpha == True & is_digit == False & is_punct == False")
             .sort_values(by=['pmid', 'token_order'])
             .groupby('pmid')
             .agg({'lemma': lambda x: ' '.join(x)})
             .reset_index())

# Check the number of documents after filtering
print(f'No. of docs in domain C: {len(c_lem_tbl)}')
print(f'No. of docs in domain A: {len(a_lem_tbl)}')

No. of docs in domain C: 1273
No. of docs in domain A: 153


In [14]:
# --------------------------------------------------------------
# TF-IDF analysis
# --------------------------------------------------------------

# Apply TF-IDF vectorization to identify significant bigrams in the text.
# N-gram range is set to (2, 2) for bigrams, and minimum document frequency is 2.
c_vec = TfidfVectorizer(min_df=2, ngram_range=(2,2))
c_ngrams = c_vec.fit_transform(c_lem_tbl['lemma'])
c_cnt = c_ngrams.toarray().sum(axis=0)
# Extract the ngrams and their frequencies into a DataFrame
c_vocab = c_vec.vocabulary_
c_df_ngram = pd.DataFrame(sorted([(k, c_cnt[i]) for k, i in c_vocab.items()]), columns=['ngram', 'freq'])

# Repeat the process for the A-domain
a_vec = TfidfVectorizer(min_df=2, ngram_range=(2,2))
a_ngrams = a_vec.fit_transform(a_lem_tbl['lemma'])
a_cnt = a_ngrams.toarray().sum(axis=0)
a_vocab = a_vec.vocabulary_
a_df_ngram = pd.DataFrame(sorted([(k, a_cnt[i]) for k, i in a_vocab.items()]), columns=['ngram', 'freq'])

In [15]:
# --------------------------------------------------------------
# Identifying shared intermediate terms (b-terms)
# --------------------------------------------------------------

# Shared intermediate terms (b-terms) are identified as the intersection
# of significant ngrams in both domains.
c_ngram_lst = c_df_ngram['ngram'].to_list()
a_ngram_lst = a_df_ngram['ngram'].to_list()
b_terms = set(c_ngram_lst) & set(a_ngram_lst)

print(f"Shared b-terms: {b_terms}")

Shared b-terms: {'blood pressure', 'long term', 'platelet aggregation', 'blood viscosity', 'platelet function', 'myocardial infarction'}


In [16]:
# --------------------------------------------------------------
# Borda count for rank aggregation
# --------------------------------------------------------------

# Implement the Borda Count method to aggregate ranks of b-terms from
# both C and A domains based on their frequencies.

def borda_count(R, v=None):
    """
    Borda count rank aggregation method.

    Parameters:
        R (numpy.ndarray): Matrix with rankings.
        v (list or numpy.ndarray): Vector of votes for each ranking. Default is equal weights.

    Returns:
        tuple: A tuple containing:
            - r (numpy.ndarray): Vector with aggregated ranking.
            - w (numpy.ndarray): Vector with weighted scores.
    """
    m, n = R.shape
    
    # Assign equal votes if `v` is not provided
    if v is None:
        v = np.ones(n)
    
    # Initialize the weights matrix
    W = np.zeros((m, n))
    
    # Populate the weights matrix based on rankings
    for i in range(n):
        r = R[:, i]
        w = np.zeros(m)
        # Assign scores from m to 1 based on descending order
        w[np.argsort(-r)] = np.arange(m, 0, -1)
        W[:, i] = w
    
    # Compute the weighted scores
    w = W @ v
    
    # Aggregate the rankings
    r = np.zeros(m, dtype=int)
    r[np.argsort(-w)] = np.arange(1, m + 1)
    
    return r, w

# Calculate the ranks and weights for shared b-terms
c_df_rank = c_df_ngram.query('ngram in @b_terms')
a_df_rank = a_df_ngram.query('ngram in @b_terms')

rank_df = (pd.merge(c_df_rank, a_df_rank, on='ngram')
           .set_axis(['ngram', 'freq_c', 'freq_a'], axis=1)
           .assign(rank_c = lambda x: x.freq_c.rank(ascending=False))
           .assign(rank_a = lambda x: x.freq_a.rank(ascending=False))
          )

# Perform Borda count rank aggregation
R = np.array(rank_df[['rank_c', 'rank_a']])
bc_r, bc_w = borda_count(R)

# Sort the b-terms based on count output
b_terms_rnk = rank_df.loc[bc_r - 1]
print(b_terms_rnk)

                   ngram    freq_c    freq_a  rank_c  rank_a
1        blood viscosity  6.436829  1.029254     2.0     5.0
3  myocardial infarction  2.787341  1.308833     4.0     3.0
2              long term  8.008717  1.024821     1.0     6.0
4   platelet aggregation  1.772796  1.472776     6.0     2.0
0         blood pressure  3.736304  1.304103     3.0     4.0
5      platelet function  1.796438  1.474065     5.0     1.0
